<a href="https://colab.research.google.com/github/diegorusso/deep-learning-scratch-course/blob/master/02_regressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>